In [10]:
# Blood Transfusion Service Center
# raw data looks like:
# 2,50,12500,98,1
# 1 stand for donating blood; 0 stands for not donating blood

import numpy as np
import tensorflow.keras as K

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'  # suppress CPU msg

class MyLogger(K.callbacks.Callback):
  def __init__(self, n):
    self.n = n   # print loss & acc every n epochs

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.n == 0:
      curr_loss =logs.get('loss')
      curr_acc = logs.get('accuracy') 
      print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % \
        (epoch, curr_loss, curr_acc))

def main():
  print("\nBlood Transfusion Service Center dataset example \n")
  np.random.seed(1)

  # 1. load data
  # print("Loading data into memory ")
  train_file = "C:/Users/Ioana/Desktop/IA/transfusion_train.data"
  test_file = "C:/Users/Ioana/Desktop/IA/transfusion_test.data"

  train_x_nenorm = np.loadtxt(train_file, delimiter=',', usecols=[0,1,2,3], dtype=np.float32)
  col0_norm = (train_x_nenorm[:,0]-min(train_x_nenorm[:,0])) / (max(train_x_nenorm[:,0])-min(train_x_nenorm[:,0]))
  col1_norm = (train_x_nenorm[:,1]-min(train_x_nenorm[:,1])) / (max(train_x_nenorm[:,1])-min(train_x_nenorm[:,1]))
  col2_norm = (train_x_nenorm[:,2]-min(train_x_nenorm[:,2])) / (max(train_x_nenorm[:,2])-min(train_x_nenorm[:,2]))
  col3_norm = (train_x_nenorm[:,3]-min(train_x_nenorm[:,3])) / (max(train_x_nenorm[:,3])-min(train_x_nenorm[:,3]))
  train_x=np.stack((col0_norm,col1_norm,col2_norm,col3_norm),axis=1)
  train_y = np.loadtxt(train_file, delimiter=',',
    usecols=[4], dtype=np.float32)
  test_x_nenorm = np.loadtxt(test_file, delimiter=',', 
    usecols=[0,1,2,3], dtype=np.float32)
  col0_norm1 = (test_x_nenorm[:,0]-min(test_x_nenorm[:,0])) / (max(test_x_nenorm[:,0])-min(test_x_nenorm[:,0]))
  col1_norm1 = (test_x_nenorm[:,1]-min(test_x_nenorm[:,1])) / (max(test_x_nenorm[:,1])-min(test_x_nenorm[:,1]))
  col2_norm1 = (test_x_nenorm[:,2]-min(test_x_nenorm[:,2])) / (max(test_x_nenorm[:,2])-min(test_x_nenorm[:,2]))
  col3_norm1 = (test_x_nenorm[:,3]-min(test_x_nenorm[:,3])) / (max(test_x_nenorm[:,3])-min(test_x_nenorm[:,3]))
  test_x=np.stack((col0_norm1,col1_norm1,col2_norm1,col3_norm1),axis=1)
  test_y =np.loadtxt(test_file, delimiter=',',
    usecols=[4], dtype=np.float32)

  # 2. define 4-(x-x)-1 deep NN model
  print("Creating 4-(8-8)-1 binary NN classifier \n")
  my_init = K.initializers.glorot_uniform(seed=1)
  model = K.models.Sequential()
  model.add(K.layers.Dense(units=8, input_dim=4,
    activation='tanh', kernel_initializer=my_init)) 
  model.add(K.layers.Dense(units=8, activation='tanh',
    kernel_initializer=my_init)) 
  model.add(K.layers.Dense(units=1, activation='sigmoid',
    kernel_initializer=my_init)) 

  # 3. compile model
  simple_sgd = K.optimizers.SGD(learning_rate=0.01)  
  model.compile(loss='binary_crossentropy',
    optimizer=simple_sgd, metrics=['accuracy'])  

  # 4. train model
  max_epochs = 5000
  my_logger = MyLogger(n=250)
  h = model.fit(train_x, train_y, batch_size=32,
    epochs=max_epochs, verbose=0, callbacks=[my_logger]) 

  # 5. evaluate model
  np.set_printoptions(precision=4, suppress=True)
  eval_results = model.evaluate(test_x, test_y, verbose=0) 
  print("\nLoss, accuracy on test data: ")
  print("%0.4f %0.2f%%" % (eval_results[0], \
eval_results[1]*100))

  # 6. save model
  # mp = ".\\Models\\banknote_model.h5"
  # model.save(mp)

  # 7. make a prediction
  inpts = np.array([[0.5,0.5,0.5,0.5]], dtype=np.float32)
  pred = model.predict(inpts)
  print("\nPredicting authenticity for: ")
  print(inpts)
  print("Probability that class = 1 ( he/she donated blood in March 2007):")
  print(pred)

if __name__=="__main__":
  main()



Blood Transfusion Service Center dataset example 

Creating 4-(8-8)-1 binary NN classifier 

epoch =    0  loss = 0.620762  acc = 0.76%
epoch =  250  loss = 0.474905  acc = 0.77%
epoch =  500  loss = 0.471545  acc = 0.78%
epoch =  750  loss = 0.471464  acc = 0.78%
epoch = 1000  loss = 0.470742  acc = 0.78%
epoch = 1250  loss = 0.470404  acc = 0.78%
epoch = 1500  loss = 0.470032  acc = 0.79%
epoch = 1750  loss = 0.468633  acc = 0.79%
epoch = 2000  loss = 0.467279  acc = 0.79%
epoch = 2250  loss = 0.465817  acc = 0.80%
epoch = 2500  loss = 0.464320  acc = 0.79%
epoch = 2750  loss = 0.463318  acc = 0.79%
epoch = 3000  loss = 0.462272  acc = 0.79%
epoch = 3250  loss = 0.461578  acc = 0.79%
epoch = 3500  loss = 0.461547  acc = 0.79%
epoch = 3750  loss = 0.461139  acc = 0.79%
epoch = 4000  loss = 0.460199  acc = 0.79%
epoch = 4250  loss = 0.460235  acc = 0.79%
epoch = 4500  loss = 0.459801  acc = 0.80%
epoch = 4750  loss = 0.459402  acc = 0.79%

Loss, accuracy on test data: 
0.5370 73.30%

